In [ ]:
import pyspark
import sys
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('week7').getOrCreate()
path='/content/kddcup.data_10_percent_corrected'
data=spark.read.csv(path,inferSchema=True)
col_names = ["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent",
                "hot","num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root",
                "num_file_creations","num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login",
                "count","srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
                "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count","dst_host_same_srv_rate",
                "dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate",
                "dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]
data=data.toDF(*col_names)
data.show(5)
data.printSchema()
data.groupby('label').count().show()
categorical=[]
numerical=[]
for col,coltype in data.dtypes:
  if coltype=='string':
    categorical.append(col)
  else:
    numerical.append(col)
from pyspark.ml.feature import VectorAssembler
output='featureVector'
assembler=VectorAssembler(inputCols=numerical,outputCol=output)
data=assembler.transform(data)
data.select(["featureVector","label"]).show()
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="featureVector",outputCol="scaledFeatures",withStd=True,withMean=True)
scaler_model = scaler.fit(data)
data = scaler_model.transform(data)
data.select('featureVector','scaledFeatures').show()
df=data.drop("protocol_type", "service", "flag")
from pyspark.ml.clustering import KMeans, KMeansModel
trainErr=[]
for k in list(range(60, 271, 30)):
  kmeans = KMeans().setK(k).setPredictionCol("cluster").setFeaturesCol("featureVector")
  model=kmeans.fit(df)
  trainErr.append(model.summary.trainingCost)
j=0
dictErr={}
for k in list(range(60, 271, 30)):
  print(k,trainErr[j])
  dictErr[trainErr[j]]=k
  j+=1
k = 270
# Define KMeans model
kmeans = KMeans().setK(k).setPredictionCol("cluster").setFeaturesCol("featureVector")
# Fit KMeans model to data
model = kmeans.fit(df)
transformed_data = model.transform(df).select("cluster", "label")
count_by_cluster_label = transformed_data.groupBy("cluster", "label").count().orderBy("cluster", "label")
count_by_cluster_label.show()